## Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto
### Phase1: Load the Postal Codes in a DataFrame

In [63]:
import pandas as pd
import qgrid
import urllib.request
import pandas as pd


# function to print out dataframe
def showQgrid(df):
    from IPython.display import display
    grid = qgrid.QGridWidget(df=df)
    QGridWidget = qgrid.show_grid(df.head(), show_toolbar=True)
    QGridWidget.get_changed_df()
    display(grid)

In [8]:
#Load Desired Table in Data Frame
WikiData = urllib.request.urlopen("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").read()
WikiTables = pd.read_html(WikiData)
df_post = WikiTables[0]

#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df_post_table = df_post[ df_post['Borough'] != "Not assigned"]

## Solution 1 In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [10]:
df_post_table.shape

(103, 3)

In [9]:
df_post_table

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


### Phase2: Load the Locations in a DataFrame    

In [12]:
GeoData = pd.read_csv("https://cocl.us/Geospatial_data")

In [13]:
df_loc = pd.merge(df_post,GeoData,left_on='Postal Code',right_on='Postal Code')

## Solution 2 : Print out the Postal Code Dataframe

In [14]:
df_loc

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


### Phase3: Cluster the locations in Toronto

In [15]:
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

In [16]:
address = 'Toronto'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
#print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))


# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df_loc['Latitude'], df_loc['Longitude'], df_loc['Borough'], df_loc['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### 3.1 Retrieve Venue Details from Foursquare

In [17]:
CLIENT_ID = 'UAKLSDGE4CSMD3AA53HLK524Y4ELAS00ATYDLXY41QJWMDCK' # your Foursquare ID
CLIENT_SECRET = 'IQYOKO3UWLKNKIHF3JAFDNE5SPPFWL2IIEN1BJ53GK34GYVC' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    LIMIT = 100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Toronto_venues = getNearbyVenues(names=df_loc['Neighbourhood'],
                                   latitudes=df_loc['Latitude'],
                                   longitudes=df_loc['Longitude']
                                  )

In [20]:
Toronto_venues.shape

(2162, 7)

In [21]:
Toronto_venues

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Sun Life,43.754760,-79.332783,Construction & Landscaping
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
...,...,...,...,...,...,...,...
2157,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Royal Canadian Legion #210,43.628855,-79.518903,Social Club
2158,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Islington Florist & Nursery,43.630156,-79.518718,Flower Shop
2159,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Koala Tan Tanning Salon & Sunless Spa,43.631370,-79.519006,Tanning Salon
2160,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Kingsway Boxing Club,43.627254,-79.526684,Gym


### Prepare a Dataframe with the Top 5 Neighbourhoods so that we can use for checking later

In [22]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighbourhood'] = Toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.shape

(2162, 273)

In [23]:
Toronto_grouped = Toronto_onehot.groupby('Neighbourhood').mean().reset_index()
num_top_venues = 10

for hood in Toronto_grouped['Neighbourhood']:
    #print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    #print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    #print('\n')

In [24]:
import numpy as np

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = Toronto_grouped['Neighbourhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

#neighborhoods_venues_sorted.head()
neighborhoods_venues_sorted.shape

(96, 11)

### Pivot the table so we get number of venues by type for each neighbourhood. Also Drop the Not Needed columns

In [88]:
# Step 1: Pivot it so that you get aggregates of each venue type in a row
Toronto_venues_Temp2 = Toronto_venues.drop(['Venue'],axis=1)
Toronto_venues_Group2 = pd.pivot_table(Toronto_venues_Temp2, 
                                       index=['Neighbourhood',
                                              'Neighbourhood Latitude', 
                                              'Neighbourhood Longitude',
                                              'Venue Category'],
                                       values=['Venue Latitude'], aggfunc=len,fill_value=0)
Toronto_venues_Group2.rename(columns={"Venue Latitude": "Venue_Counts"},inplace=True)
Toronto_venues_Group2.reset_index(inplace=True)

In [89]:
#showQgrid(Toronto_venues_Group2)

In [114]:
# Step 2 Put Venue types in columns, and one Neighbourhood should be each Row, values should be number of venues in that neighbourhood
Toronto_venues_Group3 = pd.pivot_table(Toronto_venues_Group2, 
                                       index=['Neighbourhood',
                                              'Neighbourhood Latitude', 
                                              'Neighbourhood Longitude'],
                                       columns=['Venue Category'],
                                        values=['Venue Category'], 
                                              aggfunc=len,fill_value=0)
Toronto_venues_Group3.rename(columns={"Venue Category": "Venue_Counts"},inplace=True)

In [115]:
Toronto_venues_Group3.columns = Toronto_venues_Group3.columns.get_level_values(1)
Toronto_venues_Group3.reset_index(inplace=True)
#Toronto_venues_Group3.columns = [' '.join(col).strip() for col in Toronto_venues_Group3.columns.values]
Toronto_venues_Group3.rename(columns={"Venue Category":"","":"Neighbourhood","":"Neighbourhood Latitude","":"Neighbourhood Longitude"},inplace=True)

In [116]:
#The Final Dataframe ready to be clustered
Toronto_venues_Group3.head()

Venue Category,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,...,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,43.794200,-79.262029,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Alderwood, Long Branch",43.602414,-79.543484,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Bayview Village,43.786947,-79.385975,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Bedford Park, Lawrence Manor East",43.733283,-79.419750,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [117]:
Toronto_venues_Group3.describe()

Venue Category,Neighbourhood Latitude,Neighbourhood Longitude,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,...,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
count,100.000000,100.000000,100.00,100.00,100.000000,100.00,100.00,100.00,100.00,100.00000,100.000000,100.00,100.000000,100.000000,100.000000,100.00000,100.000000,100.00,100.00,100.000000,100.00,100.000000,100.000000,100.000000,100.000000,100.000000,100.00,100.00,100.000000,100.00,100.00,100.000000,100.000000,100.000000,100.000000,100.000000,100.00,100.00,100.000000,100.000000,100.000000,100.000000,100.000000,100.00,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.00,100.000000,100.00000,100.000000,100.00,100.000000,100.000000,100.000000,100.000000,100.00,100.000000,100.000000,100.000000,100.00,100.00,100.00,100.00,100.000000,100.000000,100.000000,100.000000,100.00,100.00,100.000000,100.00,100.00000,100.000000,100.000000,100.00,100.000000,100.000000,100.00,100.000000,100.000000,100.00,100.000000,100.000000,100.00,100.000000,100.000000,100.000000,100.000000,100.000000,100.00,100.00000,100.00,100.000000,100.00,100.000000,100.000000
mean,43.703131,-79.397942,0.01,0.01,0.020000,0.01,0.01,0.01,0.01,0.15000,0.030000,0.01,0.130000,0.020000,0.050000,0.08000,0.050000,0.01,0.01,0.040000,0.01,0.040000,0.260000,0.220000,0.200000,0.050000,0.01,0.01,0.040000,0.01,0.01,0.110000,0.090000,0.020000,0.020000,0.050000,0.01,0.01,0.130000,0.030000,0.020000,0.190000,0.100000,0.01,0.070000,0.030000,0.120000,0.140000,0.040000,0.030000,...,0.01,0.060000,0.08000,0.320000,0.01,0.020000,0.120000,0.030000,0.100000,0.01,0.050000,0.020000,0.040000,0.01,0.01,0.01,0.01,0.090000,0.040000,0.070000,0.030000,0.01,0.01,0.100000,0.01,0.08000,0.020000,0.200000,0.01,0.020000,0.050000,0.01,0.020000,0.060000,0.01,0.160000,0.120000,0.01,0.050000,0.050000,0.030000,0.130000,0.020000,0.01,0.08000,0.01,0.100000,0.01,0.050000,0.120000
std,0.051156,0.095723,0.10,0.10,0.140705,0.10,0.10,0.10,0.10,0.35887,0.171447,0.10,0.337998,0.140705,0.219043,0.27266,0.219043,0.10,0.10,0.196946,0.10,0.196946,0.440844,0.416333,0.402015,0.219043,0.10,0.10,0.196946,0.10,0.10,0.314466,0.287623,0.140705,0.140705,0.219043,0.10,0.10,0.337998,0.171447,0.140705,0.394277,0.301511,0.10,0.256432,0.171447,0.326599,0.348735,0.196946,0.171447,...,0.10,0.238683,0.27266,0.468826,0.10,0.140705,0.326599,0.171447,0.301511,0.10,0.219043,0.140705,0.196946,0.10,0.10,0.10,0.10,0.287623,0.196946,0.256432,0.171447,0.10,0.10,0.301511,0.10,0.27266,0.140705,0.402015,0.10,0.140705,0.219043,0.10,0.140705,0.238683,0.10,0.368453,0.326599,0.10,0.219043,0.219043,0.171447,0.337998,0.140705,0.10,0.27266,0.10,0.301511,0.10,0.219043,0.326599
min,43.602414,-79.615819,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.00000,0.

In [118]:
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing

X = Toronto_venues_Group3.iloc[: ,3:273]
X.head()

Venue Category,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Café,...,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [119]:
X = preprocessing.StandardScaler().fit(X).transform(X)

### 3.3 Cluster the Neighborhoods

In [120]:
# set number of clusters
kclusters = 6

#Toronto_grouped_clustering = Toronto_venues_Group3.drop(['Neighbourhood','Neighbourhood Latitude','Neighbourhood Longitude'], 1)
#Toronto_grouped_clustering.describe(include = 'all')
pd.options.display.max_rows = 999
#Toronto_grouped_clustering.dtypes

In [121]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(X)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20] 

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 1, 0])

In [122]:
# add clustering labels
Toronto_venues_Group3.insert(0, 'Cluster Labels', kmeans.labels_)

## Solution 3 : Clustering Analysis Results

In [123]:
Toronto_merged = df_loc

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(Toronto_venues_Group3.set_index('Neighbourhood'), on='Neighbourhood')

#Toronto_merged.head() # check the last columns!
#Toronto_merged.describe(include='all')


In [124]:
Toronto_merged['Cluster Labels'] = Toronto_merged['Cluster Labels'].fillna(0)

### Distribution of Clusters

In [125]:
Toronto_merged.groupby('Cluster Labels').count()[['Neighbourhood']]

,Neighbourhood
Cluster Labels,
0.0,106
1.0,7
2.0,1
3.0,1
4.0,1
5.0,1


In [126]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighbourhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    colorlabel = int(cluster)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[colorlabel],
        fill=True,
        fill_color=rainbow[colorlabel],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Interpret the Results

In [127]:
#Toronto_venues_Group3.head()
Toronto_merged2 = neighborhoods_venues_sorted.join(Toronto_venues_Group3.set_index('Neighbourhood'), on='Neighbourhood')
#Toronto_merged2
#showQgrid(Toronto_merged2)
#Toronto_merged2.head(5)
pd.options.display.max_columns = 100

In [128]:
showQgrid(Toronto_merged2.iloc[:,0:12])

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…